In [5]:
## import auto reload
%load_ext autoreload
%autoreload 2

from simulation import run_simulation, initialize_simulation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
num_employees = 100
num_teams = 10
min_team_size = 8
num_friendships = 5
num_spaces = 20
space_types = ["Workstation"] * 10 + ["Meeting Room"] * 5 + ["Break Area"] * 3 + ["Quiet Area"] * 2
space_capacities = [5] * 10 + [10] * 5 + [15] * 3 + [8] * 2
hierarchy_levels = 5
hierarchy_adjustment_factor = 0.1

emps, spaces, team_graph, friendship_graph = initialize_simulation(num_employees, num_teams, min_team_size,
                                                                   num_friendships, num_spaces, space_types, space_capacities, hierarchy_levels, hierarchy_adjustment_factor)

run_simulation(emps, spaces, 10, initial_info_holder=1, team_graph=team_graph, friendship_graph=friendship_graph)


Time step 1
Informed employees: 1/100 (1.00%)
Space 1 (Workstation): [29, 38, 54, 78, 80, 98]
Space 2 (Workstation): [5, 9, 86]
Space 3 (Workstation): [6, 23, 42, 79]
Space 4 (Workstation): [10, 14, 33, 34, 36, 48, 85]
Space 5 (Workstation): [11, 88]
Space 6 (Workstation): [2, 13, 17, 31, 60, 68, 83, 97, 100]
Space 7 (Workstation): [20, 61, 94]
Space 8 (Workstation): [16, 43, 47, 91]
Space 9 (Workstation): [8, 19, 22, 24, 84]
Space 10 (Workstation): [44, 74, 90]
Space 11 (Meeting Room): [7, 52, 53, 56, 70]
Space 12 (Meeting Room): [45, 67, 72, 89, 95]
Space 13 (Meeting Room): [3, 27, 39, 51, 93]
Space 14 (Meeting Room): [18, 21, 25, 77, 92, 96]
Space 15 (Meeting Room): [4, 62, 99]
Space 16 (Break Area): [12, 15, 59, 65]
Space 17 (Break Area): [1, 28, 30, 32, 49, 58, 66, 75, 82, 87]
Space 18 (Break Area): [41, 69]
Space 19 (Quiet Area): [26, 37, 40, 50, 57, 63, 64, 71]
Space 20 (Quiet Area): [35, 46, 55, 73, 76, 81]
Information spread: [1]

Time step 2
Informed employees: 1/100 (1.00%)
